In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import osmnx as ox
import pyro
import pyro.contrib.gp as gp
import pyro.distributions as dist
import sklearn as sk
from pyro.infer import MCMC, NUTS, Predictive,HMC
import torch
import random
import kennard_stone as ks
import calendar
import arviz as az
import jax.numpy as jnp
from scipy.optimize import curve_fit
import seaborn as sns
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap, factor_mark
import numpy as np
from bokeh.io import curdoc
from bokeh.layouts import column
from bokeh.models import Div
from bokeh.palettes import Spectral
from print_versions import print_versions


In [2]:
print_versions(globals())

json==2.0.9
ipykernel==6.28.0
numpy==1.26.4
pandas==2.2.3
tensorflow==2.18.0
keras==3.5.0
torch==2.5.1
ipywidgets==7.6.5
xarray==2023.6.0
osmnx==1.9.3
pyro==1.9.1
sklearn==1.2.2
kennard_stone==3.0.0
arviz==0.20.0
scipy==1.14.0
seaborn==0.12.2
bokeh==3.6.0


In [5]:

PM25=pd.read_excel(r"pm25.xlsx")
PM25=PM25.loc[PM25.pm_final.isna()==0,:]

In [6]:
X_full, y_full=PM25.loc[:,["sv_final","flow"]],PM25.loc[:,["pm_final","sv_final"]]

In [7]:
X_train, X_test, y_train, y_test = ks.train_test_split(X_full, y_full, test_size = 0.2)

2025-01-31 09:56:37,147 - kennard_stone.utils._pairwise:109[INFO] - Calculating pairwise distances using scikit-learn.

2025-01-31 09:56:37,177 - kennard_stone.utils._pairwise:109[INFO] - Calculating pairwise distances using scikit-learn.



In [8]:
traning_pm25 = {"X":torch.tensor(X_train.flow.values).float(),"y":torch.tensor(y_train.pm_final.values).float()}
traning_BC = {"X":torch.tensor(X_train.flow.values).float(),"y":torch.tensor(y_train.sv_final.values).float()}
test_pm25 = {"X":torch.tensor(X_test.flow.values).float(),"y":torch.tensor(y_test.pm_final.values).float()}

In [13]:
def plot(
    plot_observed_data=False,
    plot_predictions=False,
    n_prior_samples=0,
    model=None,
    kernel=None,
    n_test=500,
    ax=None,
):

    if ax is None:
        fig, ax = plt.subplots(figsize=(12, 6))
    if plot_observed_data:
        ax.plot(X.numpy(), y.numpy(), "kx")
    if plot_predictions:
        Xtest = torch.linspace(-0.5, 5.5, n_test)  # test inputs
        # compute predictive mean and variance
        with torch.no_grad():
            if type(model) == gp.models.VariationalSparseGP:
                mean, cov = model(Xtest, full_cov=True)
            else:
                mean, cov = model(Xtest, full_cov=True, noiseless=False)
        sd = cov.diag().sqrt()  # standard deviation at each input point x
        ax.plot(Xtest.numpy(), mean.numpy(), "r", lw=2)  # plot the mean
        ax.fill_between(
            Xtest.numpy(),  # plot the two-sigma uncertainty about the mean
            (mean - 2.0 * sd).numpy(),
            (mean + 2.0 * sd).numpy(),
            color="C0",
            alpha=0.3,
        )
    if n_prior_samples > 0:  # plot samples from the GP prior
        Xtest = torch.linspace(-0.5, 5.5, n_test)  # test inputs
        noise = (
            model.noise
            if type(model) != gp.models.VariationalSparseGP
            else model.likelihood.variance
        )
        cov = kernel.forward(Xtest) + noise.expand(n_test).diag()
        samples = dist.MultivariateNormal(
            torch.zeros(n_test), covariance_matrix=cov
        ).sample(sample_shape=(n_prior_samples,))
        ax.plot(Xtest.numpy(), samples.numpy().T, lw=2, alpha=0.4)

    ax.set_xlim(-0.5, 5.5)

In [16]:
X,y=torch.tensor(X_train.flow.values).float(),torch.tensor(y_train.pm_final.values).float()

In [ ]:
pyro.clear_param_store()
rbf = gp.kernels.RBF(input_dim=1)

rbf.variance = pyro.nn.PyroSample(dist.HalfNormal(torch.tensor(10.)))

rbf.lengthscale = pyro.nn.PyroSample(dist.LogNormal(torch.tensor(0.), torch.tensor(0.4)))
gpr = gp.models.GPRegression(X,y, rbf)
gpr.noise = pyro.nn.PyroSample(dist.HalfNormal(torch.tensor(20.)))

nuts_kernel = NUTS(gpr.model,target_accept_prob=0.7)

mcmc = MCMC(nuts_kernel, num_samples=1000,warmup_steps=1500,num_chains=1)

mcmc.run()

Warmup:  32%|████████████▊                           | 803/2500 [02:27,  8.54it/s, step size=1.06e+00, acc. prob=0.693]

In [ ]:
plot(model=gpr, plot_observed_data=True, plot_predictions=True)
plt.xlim(3.3,4.6)
plt.savefig("flow_pm25.png")

In [ ]:
mcmc.diagnostics()

In [ ]:
posterior_samples = mcmc.get_samples()
posterior_predictive = Predictive(gpr, posterior_samples)(traning_pm25["X"])
prior = Predictive(gpr, num_samples=500)(traning_pm25["X"])

pyro_data = az.from_pyro(mcmc,
    prior=prior,
    posterior_predictive=posterior_predictive,log_likelihood=False

)
pyro_data

In [ ]:
az.style.use("arviz-doc")

ax = az.plot_trace(pyro_data)

In [ ]:
ax = az.plot_density(
    [pyro_data],
    shade=0.1,
)

In [ ]:
torch.save(gpr, "PM2.5_flow_correction")

In [ ]:
exp_curve=curve_fit(lambda t,a,b: a*np.exp(b*t),  X_train.flow,y_train.pm_final)
linear_curve=curve_fit(lambda t,a,b: a * t +b,  X_train.flow,y_train.pm_final)
log_curve=curve_fit(lambda t,a,b: a * t +b,  X_train.flow,np.log(y_train.pm_final))

In [ ]:
def exp(x, a, b):
    return a * np.exp(b * x)
def linear(x, a, b):
    return a * x +b


In [ ]:
y_train["PM25_Gaussion"]=(y*gpr(torch.tensor(4*np.ones(len(X))).float())[0]/gpr(X)[0]).detach().numpy()
y_train["pm25_linear"]=linear(4,*linear_curve[0])/linear(X_train.flow,*linear_curve[0])*y_train.pm_final
y_train["pm25_exp"]=exp(4,*exp_curve[0])/exp(X_train.flow,*exp_curve[0])*y_train.pm_final
y_train["pm25_log"]=linear(4,*log_curve[0])/linear(X_train.flow,*log_curve[0])*y_train.pm_final

In [ ]:
x=np.linspace(2,5,200)
X_tor=torch.tensor(x).float()
data=pd.DataFrame(x,index=np.arange(100),columns=["flow"])
data["lieanr_scale_pm"]=(linear(4,*linear_curve[0])/linear(x,*linear_curve[0]))
data["exp_scale_pm"]=exp(4,*exp_curve[0])/exp(x,*exp_curve[0])
data["guassion_scale_pm"]=(gpr(torch.tensor(4*np.ones(len(x))).float())[0]/gpr(X_tor)[0]).detach().numpy()
data["log_scale_pm"]=linear(4,*log_curve[0])/linear(x,*log_curve[0])

In [ ]:
pyro.clear_param_store()
rbf_bc = gp.kernels.RBF(input_dim=1)

rbf_bc.variance = pyro.nn.PyroSample(dist.HalfNormal(torch.tensor(5.)))
X=traning_BC["X"]
y=traning_BC["y"]
rbf_bc.lengthscale = pyro.nn.PyroSample(dist.LogNormal(torch.tensor(0.), torch.tensor(0.4)))
gpr_bc = gp.models.GPRegression(X,y, rbf)
gpr_bc.noise = pyro.nn.PyroSample(dist.HalfNormal(torch.tensor(5.)))


nuts_kernel_bc = NUTS(gpr_bc.model)


mcmc_bc = MCMC(nuts_kernel_bc, num_samples=500,warmup_steps=1000)

mcmc_bc.run()

In [ ]:
posterior_samples_bc = mcmc_bc.get_samples(500)
posterior_predictive_bc = Predictive(gpr_bc, posterior_samples_bc)(traning_BC["X"])
prior_bc = Predictive(gpr_bc, num_samples=500)(traning_BC["X"])

pyro_data_bc = az.from_pyro(mcmc_bc,
    prior=prior_bc,
    posterior_predictive=posterior_predictive_bc,

)
pyro_data

In [ ]:
az.style.use("arviz-doc")

ax = az.plot_trace(pyro_data_bc)

In [ ]:
ax = az.plot_density(
    [pyro_data_bc],
    shade=0.1,
)

In [ ]:
torch.save(gpr_bc, "BC_flow_correction")

In [ ]:
plot(model=gpr_bc, plot_observed_data=True, plot_predictions=True)
plt.xlim(3.25,4.7)

In [ ]:
exp_curve_bc=curve_fit(lambda t,a,b: a*np.exp(b*t),  X_train.flow,y_train.sv_final)
linear_curve_bc=curve_fit(lambda t,a,b: a * t +b,  X_train.flow,y_train.sv_final)
log_curve_bc=curve_fit(lambda t,a,b: a * t +b,  X_train.flow,np.log(y_train.sv_final))

In [ ]:
y_train["BC_Gaussion"]=(y *gpr_bc(torch.tensor(4*np.ones(len(X))).float())[0]/gpr_bc(X)[0]).detach().numpy()
y_train["BC_linear"]=linear(4,*linear_curve_bc[0])/linear(X_train.flow,*linear_curve_bc[0])*y_train.sv_final
y_train["BC_exp"]=exp(4,*exp_curve_bc[0])/exp(X_train.flow,*exp_curve_bc[0])*y_train.sv_final
y_train["BC_log"]=linear(4,*log_curve_bc[0])/linear(X_train.flow,*log_curve_bc[0])*y_train.sv_final

In [ ]:

data["lieanr_scale_bc"]=(linear(4,*linear_curve_bc[0])/linear(x,*linear_curve_bc[0]))
data["exp_scale_bc"]=exp(4,*exp_curve_bc[0])/exp(x,*exp_curve_bc[0])
data["guassion_scale_bc"]=(gpr_bc(torch.tensor(4*np.ones(len(x))).float())[0]/gpr_bc(X_tor)[0]).detach().numpy()
data["log_scale_bc"]=linear(4,*log_curve_bc[0])/linear(x,*log_curve_bc[0])

In [ ]:
y_train

In [ ]:
p = figure(title = "Penguin size", background_fill_color="#fafafa")

p.line(data.guassion_scale_pm, data.guassion_scale_bc, line_color="blue",
        line_width=3, legend_label="Linear")
show(p)

In [ ]:
p = figure(title = r"$$PM_{2.5}$$", background_fill_color="#fafafa")

p.line(data.flow, data.lieanr_scale_pm, line_color="blue", line_width=3, legend_label="Linear")
p.line(data.flow, data.exp_scale_pm, line_color="red", line_width=3, legend_label="Exponential")
p.line(data.flow, data.guassion_scale_pm, line_color="green",line_width=3, legend_label="Gaussian")
p.line(data.flow, data.log_scale_pm, line_color="yellow", line_width=3, legend_label="Log")
p.legend.location = "top_left"
p.legend.title = r"Correction"
p.legend.title_text_font_size = "16px"
p.legend.title_text_font_style = "bold"
p.xaxis.axis_label = r'$$Flow \frac{L}{s}$$'
p.yaxis.axis_label = 'scale'
p.xaxis.axis_label_text_font_size = "14px"
p.yaxis.axis_label_text_font_size = "16px"
p.yaxis.axis_label_text_font_style = "bold"
p.xaxis.axis_label_text_font_style = "bold"

p1 = figure(title = "Black Carbon", background_fill_color="#fafafa")

p1.line(data.flow, data.lieanr_scale_bc, line_color="blue",
        line_width=3, legend_label="Linear")
p1.line(data.flow, data.exp_scale_bc, line_color="red", line_width=3, legend_label="Exponential")
p1.line(data.flow, data.guassion_scale_bc, line_color="green", line_width=3, legend_label="Gaussian")
#p1.line(data.flow, data.log_scale_bc, line_color="yellow", line_width=3, legend_label="Log")
p1.legend.location = "top_left"
p1.legend.title = "Correction"
p1.legend.title_text_font_size = "16px"
p1.xaxis.axis_label_text_font_size = "14px"
p1.yaxis.axis_label_text_font_size = "16px"
p1.yaxis.axis_label_text_font_style = "bold"
p1.xaxis.axis_label_text_font_style = "bold"

p1.legend.title_text_font_style = "bold"
p1.xaxis.axis_label = r'$$Flow \frac{L}{s}$$'
p1.yaxis.axis_label = 'scale'
grid =  gridplot([[p, p1]], width=500, height=500)

show(column(grid))

In [ ]:
from bokeh.models import Slope

In [ ]:
"Slope: "+str(round(slope,1))+" "+"Intercept: "+str(round(intercept,1))

In [ ]:
p = figure(title = "Penguin size", background_fill_color="#fafafa")

p.scatter(y_train.pm_final, y_train.PM25_Gaussion, line_color="blue",legend_label="Linear")
slope, intercept=np.polyfit(y_train.pm_final, y_train.PM25_Gaussion,1)
slope = Slope(gradient=slope, y_intercept=intercept,
              line_color="blue", line_dash='dashed', line_width=4)

p.add_layout(legend_label="Slope: "+str(round(slope,1))+" "+"Intercept: "+str(round(intercept,1)))
p.legend.location = "top_left"
p.legend.title = "Species"

p.xaxis.axis_label = r'$$Flow \frac{\mu}{m^3}$$'
p.yaxis.axis_label = 'Scaleing point'
show(p)

In [ ]:
PM25.to_excel("pm25_BC_corrected.xlsx")

In [ ]:
slope, intercept=np.polyfit(y_train.pm_final, y_train.PM25_Gaussion,1)

In [ ]:
slope, intercept

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(10,5))
y_train.PM25_Gaussion.plot.kde(ax=ax[0],label="PM Gaussion correction")
y_train.pm_final.plot.kde(ax=ax[0],label="PM Raw meassurement")
y_train.BC_Gaussion.plot.kde(ax=ax[1],label="BC gaussion correction")
y_train.sv_final.plot.kde(ax=ax[1],label="BC meassurement")
ax[0].legend(loc=1,fontsize=9)
ax[1].legend(loc=1,fontsize=9)